Notebook created: 2018-02-06 23:30:24  
Generated from: _build_jl/jl/optgrowth.rst  

In [2]:
#=

@authors : Spencer Lyon, John Stachurski

=#

using QuantEcon
using Optim


"""
The approximate Bellman operator, which computes and returns the
updated value function Tw on the grid points.  An array to store
the new set of values Tw is optionally supplied (to avoid having to
allocate new arrays at each iteration).  If supplied, any existing data in 
Tw will be overwritten.

#### Arguments

`w` : Vector
      The value of the input function on different grid points
`grid` : Vector
         The set of grid points
`β` : AbstractFloat
         The discount factor
`u` : Function
      The utility function
`f` : Function
      The production function
`shocks` : Vector
           An array of draws from the shock, for Monte Carlo integration (to
           compute expectations).
`Tw` : Vector, optional (default=similar(w))
       Array to write output values to
`compute_policy` : Bool, optional (default=false)
                   Whether or not to compute policy function

"""
function bellman_operator(w::Vector, 
                          grid::Vector,
                          β::AbstractFloat, 
                          u::Function, 
                          f::Function, 
                          shocks::Vector, 
                          Tw::Vector = similar(w);
                          compute_policy::Bool = false)

    # === Apply linear interpolation to w === #
    w_func = LinInterp(grid, w)

    if compute_policy
        σ = similar(w)
    end

    # == set Tw[i] = max_c { u(c) + β E w(f(y  - c) z)} == #
    for (i, y) in enumerate(grid)
        objective(c) = - u(c) - β * mean(w_func.(f(y - c) .* shocks))
        res = optimize(objective, 1e-10, y)

        if compute_policy
            σ[i] = res.minimizer
        end
        Tw[i] = - res.minimum
    end

    if compute_policy
        return Tw, σ
    else
        return Tw
    end
end

bellman_operator

In [3]:
α = 0.4
β = 0.96
μ = 0
s = 0.1

c1 = log(1 - α * β) / (1 - β)
c2 = (μ + α * log(α * β)) / (1 - α)
c3 = 1 / (1 - β)
c4 = 1 / (1 - α * β)

# Utility 
u(c) = log(c)

u_prime(c) = 1 / c

# Deterministic part of production function
f(k) = k^α

f_prime(k) = α * k^(α - 1)

# True optimal policy
c_star(y) = (1 - α * β) * y

# True value function
v_star(y) = c1 + c2 * (c3 - c4) + c4 * log(y)

v_star (generic function with 1 method)

In [4]:
grid_max = 4         # Largest grid point
grid_size = 200      # Number of grid points
shock_size = 250     # Number of shock draws in Monte Carlo integral

grid_y = collect(linspace(1e-5, grid_max, grid_size))
shocks = exp.(μ + s * randn(shock_size))

250-element Array{Float64,1}:
 0.917891
 0.933462
 1.03628 
 0.9069  
 0.952874
 0.915149
 0.987194
 1.05626 
 1.04416 
 1.00311 
 0.950161
 0.867759
 1.06706 
 ⋮       
 0.98801 
 1.37926 
 1.06252 
 0.991875
 0.820841
 0.958081
 1.03711 
 0.978666
 0.837354
 1.02482 
 0.989471
 0.956911

In [7]:
using Plots
gr()

Plots.GRBackend()

In [11]:
w = bellman_operator(v_star.(grid_y),
                     grid_y,
                     β,
                     log,
                     k -> k^α,
                     shocks)

plot(grid_y, w, lw=2, alpha=0.6, label="Tv^*")
plot!(grid_y, v_star.(grid_y), lw=2, alpha=0.6, label="v^")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 -50 
 
 
 -45 
 
 
 -40 
 
 
 -35 
 
 
 -30 
 
 
 -25 
 
<polyline clip-path="url(#clip3302)" style="stroke:#009af9; stroke-width:2; stroke-opacity:0.6; fill:none" points="
 36.1567,375.869 38.8912,129.077 41.6256,113.714 44.3601,104.73 47.0946,98.3566 49.829,93.4124 52.5635,89.3732 55.298,85.9583 58.0324,82.9998 60.7669,80.3906 
 63.5014,78.0569 66.2358,75.9451 68.9703,74.0179 71.7047,72.2448 74.4392,70.603 77.1737,69.0748 79.9081,67.6451 82.6426,66.3022 85.3771,65.0362 88.1115,63.8384 
 90.846,62.7023 93.5805,61.6214 96.3149,60.591 99.0494,59.6064 101.784,58.6636 104.518,57.7596 107.253,56.8906 109.987,56.0547 112.722,55.2491 115.456,54.4717 
 118.191,53.721 120.925,52.9945 123.66,52.2913 126.394,51.6097 129.128,50.9484 131.863,50.3064 134.597,49.6823 137.332,49.0755 140.066,48.4848 142.801,47.9093 
 145.535,47.3487 148.27,46.8017 151.004,46.2678 153.739,45.7467 156.473,45.2375 159.208,44.7397 161.942,44.2528 164.677,43.7765 167.411,43.3102 170.145,42.8534 
 172.88,42.4059 175.614,41.9673 178.349,41.5372 181.083,41.1152 183.818,40.7013 186.552,40.2948 189.287,39.8957 192.021,39.5036 194.756,39.1184 197.49,38.7398 
 200.225,38.3675 202.959,38.0014 205.693,37.6412 208.428,37.2868 211.162,36.938 213.897,36.5946 216.631,36.2564 219.366,35.9233 222.1,35.5952 224.835,35.2718 
 227.569,34.9531 230.304,34.6389 233.038,34.3291 235.773,34.0236 238.507,33.7222 241.241,33.4249 243.976,33.1315 246.71,32.842 249.445,32.5562 252.179,32.274 
 254.914,31.9954 257.648,31.7202 260.383,31.4485 263.117,31.18 265.852,30.9147 268.586,30.6525 271.321,30.3935 274.055,30.1375 276.79,29.8843 279.524,29.634 
 282.258,29.3865 284.993,29.1418 287.727,28.8997 290.462,28.6603 293.196,28.4233 295.931,28.1889 298.665,27.957 301.4,27.7275 304.134,27.5003 306.869,27.2754 
 309.603,27.0528 312.338,26.8324 315.072,26.6142 317.806,26.3981 320.541,26.1841 323.275,25.9722 326.01,25.7622 328.744,25.5542 331.479,25.3481 334.213,25.144 
 336.948,24.9418 339.682,24.7413 342.417,24.5426 345.151,24.3457 347.886,24.1506 350.62,23.9572 353.354,23.7654 356.089,23.5753 358.823,23.3867 361.558,23.1998 
 364.292,23.0145 367.027,22.8307 369.761,22.6484 372.496,22.4675 375.23,22.2882 377.965,22.1102 380.699,21.9338 383.434,21.7587 386.168,21.585 388.903,21.4126 
 391.637,21.2415 394.371,21.0718 397.106,20.9034 399.84,20.7362 402.575,20.5703 405.309,20.4056 408.044,20.2421 410.778,20.0799 413.513,19.9188 416.247,19.7589 
 418.982,19.6001 421.716,19.4425 424.451,19.2859 427.185,19.1305 429.919,18.9761 432.654,18.8228 435.388,18.6706 438.123,18.5194 440.857,18.3693 443.592,18.2201 
 446.326,18.072 449.061,17.9248 451.795,17.7786 454.53,17.6333 457.264,17.489 459.999,17.3457 462.733,17.2032 465.467,17.0617 468.202,16.9211 470.936,16.7813 
 473.671,16.6425 476.405,16.5044 479.14,16.3673 481.874,16.231 484.609,16.0955 487.343,15.9609 490.078,15.8271 492.812,15.694 495.547,15.5618 498.281,15.4303 
 501.016,15.2996 503.75,15.1697 506.484,15.0406 509.219,14.9122 511.953,14.7846 514.688,14.6576 517.422,14.5314 520.157,14.4059 522.891,14.2811 525.626,14.157 
 528.36,14.0336 531.095,13.9109 533.829,13.7889 536.564,13.6675 539.298,13.5468 542.032,13.4267 544.767,13.3073 547.501,13.1885 550.236,13.0704 552.97,12.953 
 555.705,12.8361 558.439,12.7198 561.174,12.6042 563.908,12.489 566.643,12.3745 569.377,12.2607 572.112,12.1474 574.846,12.0347 577.58,11.9226 580.315,11.811 
 
 "/>
<polyline clip-path="url(#clip3302)" style="stroke:#e26f46; stroke-width:2; stroke-opacity:0.6; fill:none" points="
 36.1567,297.552 38.8912,129.077 41.6256,113.73 44.3601,104.751 47.0946,98.3803 49.829,93.4384 52.5635,89.4005 55.298,85.9865 58.0324,83.0291 60.7669,80.4205 
 63.5014,78.087 66.2358,75.976 68.9703,74.0489 71.7047,72.2761 74.4392,70.6348 77.1737,69.1067 79.9081,67.6773 82.6426,66.3345 85.3771,65.0685 88.1115,63.871 
 90.846,62.735 93.5805,61.6543 96.31

In [ ]:
w = 5 * log.(grid_y)  # An initial condition -- fairly arbitrary
n = 35

lb = "initial condition"
plot(grid_y, w, lw=2, alpha=0.6, label=lb)
for i in 1:n
    w = bellman_operator(w,
                         grid_y,
                         β,
                         log,
                         k -> k^α,
                         shocks)

    plot!(grid_y, w, color=jet(i / n), lw=2, alpha=0.6)
end

lb = "true value function"
ax[:plot](grid_y, v_star.(grid_y), "k-", lw=2, alpha=0.8, label=lb)
ax[:legend](loc="lower right")

show()

In [ ]:
function solve_optgrowth(initial_w;
                         tol::AbstractFloat=1e-6,
                         max_iter::Integer=500)

    w = initial_w  # Set initial condition
    error = tol + 1
    i = 0

    # == Create storage array for bellman_operator. Reduces  memory
    # allocation and speeds code up == #
    Tw = similar(grid_y)

    # Iterate to find solution

    while (error > tol) && (i < max_iter)
        w_new = bellman_operator(w,
                             grid_y,
                             β,
                             log,
                             k -> k^α,
                             shocks)
        error = maximum(abs, w_new - w)
        w = w_new
        i += 1
    end

    return w
end

In [ ]:
initial_w = 5 * log.(grid_y)
v_star_approx = solve_optgrowth(initial_w)

fig, ax = subplots(figsize=(9, 5))
ax[:set_ylim](-35, -24)
ax[:plot](grid_y, v_star_approx, lw=2, alpha=0.6, label="approximate value function")
ax[:plot](grid_y, v_star.(grid_y), lw=2, alpha=0.6, label="true value function")
ax[:legend](loc="lower right")
show()

In [ ]:
import QuantEcon: compute_fixed_point

Tw = similar(grid_y)
initial_w = 5 * log.(grid_y)

bellman_operator(w) = bellman_operator(w,
                                       grid_y,
                                       β,
                                       log,
                                       k -> k^α,
                                       shocks)

v_star_approx = compute_fixed_point(bellman_operator,
                                    initial_w,
                                    max_iter=500,
                                    verbose=2,
                                    print_skip=10,
                                    err_tol=1e-5)

```none
Compute iterate 10 with error 0.709153897728406
Compute iterate 20 with error 0.47095844432889145
Compute iterate 30 with error 0.3131085083453158
Compute iterate 40 with error 0.20816475495214704
Compute iterate 50 with error 0.13839472275577336
Compute iterate 60 with error 0.09200932833702069
Compute iterate 70 with error 0.061170804294146564
Compute iterate 80 with error 0.04066834706496181
Compute iterate 90 with error 0.027037644380378367
Compute iterate 100 with error 0.01797550838462314
Compute iterate 110 with error 0.011950704623018282
Compute iterate 120 with error 0.00794521845620011
Compute iterate 130 with error 0.005282240529794535
Compute iterate 140 with error 0.003511805895449527
Compute iterate 150 with error 0.0023347631704915273
Compute iterate 160 with error 0.001552226753144481
Compute iterate 170 with error 0.0010319710041208907
Compute iterate 180 with error 0.0006860880028760619
Compute iterate 190 with error 0.0004561336956570017
Compute iterate 200 with error 0.00030325256715002524
Compute iterate 210 with error 0.00020161220411551994
Compute iterate 220 with error 0.0001340383727708172
Compute iterate 230 with error 8.911308481529545e-5
Compute iterate 240 with error 5.924528747414115e-5
Compute iterate 250 with error 3.9388200878676116e-5
Compute iterate 260 with error 2.6186561168373146e-5
Compute iterate 270 with error 1.7409680953761608e-5
Compute iterate 280 with error 1.1574523867352582e-5
Converged in 284 steps
```


In [ ]:
fig, ax = subplots(figsize=(9, 5))
ax[:set_ylim](-35, -24)
ax[:plot](grid_y, v_star_approx, lw=2, alpha=0.6, label="approximate value function")
ax[:plot](grid_y, v_star.(grid_y), lw=2, alpha=0.6, label="true value function")
ax[:legend](loc="lower right")
show()

In [ ]:
Tw, σ = bellman_operator(v_star_approx,
                         grid_y,
                         β,
                         log,
                         k -> k^α,
                         shocks;
                         compute_policy=true)


cstar = (1 - α * β) * grid_y

fig, ax = subplots(figsize=(9, 5))
ax[:plot](grid_y, σ, lw=2, alpha=0.6, label="approximate policy function")
ax[:plot](grid_y, cstar, lw=2, alpha=0.6, label="true policy function")
ax[:legend](loc="lower right")
show()

In [ ]:
s = 0.05
shocks = exp.(μ + s * randn(shock_size))

In [ ]:
"""
Compute a time series given consumption policy σ.
"""
function simulate_og(σ, y0 = 0.1, ts_length=100)
    y = Array{Float64}(ts_length)
    ξ = randn(ts_length-1)
    y[1] = y0
    for t in 1:(ts_length-1)
        y[t+1] = (y[t] - σ(y[t]))^α * exp(μ + s * ξ[t])
    end
    return y
end

fig, ax = subplots(figsize=(9, 6))

for β in (0.9, 0.94, 0.98)

    Tw = similar(grid_y)
    initial_w = 5 * log.(grid_y)

    v_star_approx = compute_fixed_point(bellman_operator,
                                        initial_w,
                                        max_iter=50,
                                        verbose=0,
                                        print_skip=10,
                                        err_tol=1e-5)

    Tw, σ = bellman_operator(v_star_approx,
                             grid_y,
                             β,
                             log,
                             k -> k^α,
                             shocks,
                             compute_policy=true)

    σ_func = LinInterp(grid_y, σ)
    y = simulate_og(σ_func)
    ax[:plot](y, lw=2, alpha=0.6, label="β = $β" )
end


ax[:legend](loc="lower right")
show()